# 1.Tải Data (để sau)

In [1]:
import os
import zipfile
import urllib.request
data_dir = "/content/"

In [2]:
url = "https://download.pytorch.org/tutorial/hymenoptera_data.zip"
save_path = os.path.join(data_dir, "hymenoptera_data.zip")

if not os.path.exists(save_path):
    urllib.request.urlretrieve(url, save_path)
    
    # read by zipfile
    zip = zipfile.ZipFile(save_path)
    zip.extractall("/content/")
    zip.close()
    
    os.remove(save_path)

# 2.Tạo Dataset

## 2.1. Import thư viện và Cài đặt random seed (để sau)

In [3]:
import glob
import random
import numpy as np
import json
from PIL import Image
import matplotlib.pyplot as plt
%matplotlib inline

import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data
import torchvision
from torchvision import models, transforms
from tqdm import tqdm

In [4]:
torch.manual_seed(1234)
np.random.seed(1234)
random.seed(1234)

## 2.2. Tạo class ImageTransform

In [5]:
class ImageTransform():
  def __init__(self, resize, mean, std):
    self.data_transform = {
        'train': transforms.Compose([
          transforms.RandomResizedCrop(resize, scale=(0.5, 1)),
          transforms.RandomHorizontalFlip(),
          transforms.ToTensor(),
          transforms.Normalize(mean, std)
        ]),

        'val': transforms.Compose([
          transforms.Resize(resize),
          transforms.CenterCrop(resize),
          transforms.ToTensor(),
          transforms.Normalize(mean, std)
        ])
    }

  def __call__(self, img, phase='train'):
    return self.data_transform[phase](img)

resize = 224
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]

transform = ImageTransform(resize, mean, std)

## 2.3. Rút gọn Data chỉ còn chứa 2 folder train và val

In [6]:
def make_datapath_list(phase="train"):
  rootpath = "/content/hymenoptera_data/"
  target_path = os.path.join(rootpath+phase+"/*/*.jpg")

  path_list = []

  for path in glob.glob(target_path):
    path_list.append(path)

  return path_list

In [7]:
train_list = make_datapath_list("train")
val_list = make_datapath_list("val")

train_list[:10]

['/content/hymenoptera_data/train/ants/6743948_2b8c096dda.jpg',
 '/content/hymenoptera_data/train/ants/892108839_f1aad4ca46.jpg',
 '/content/hymenoptera_data/train/ants/1225872729_6f0856588f.jpg',
 '/content/hymenoptera_data/train/ants/1360291657_dc248c5eea.jpg',
 '/content/hymenoptera_data/train/ants/82852639_52b7f7f5e3.jpg',
 '/content/hymenoptera_data/train/ants/ant photos.jpg',
 '/content/hymenoptera_data/train/ants/841049277_b28e58ad05.jpg',
 '/content/hymenoptera_data/train/ants/175998972.jpg',
 '/content/hymenoptera_data/train/ants/484293231_e53cfc0c89.jpg',
 '/content/hymenoptera_data/train/ants/938946700_ca1c669085.jpg']

## 2.4. Chính thức tạo Dataset

In [8]:
class MyDataset(torch.utils.data.Dataset):
  def __init__(self, file_list, transform=None, phase="train"):
    self.file_list = file_list
    self.transform = transform
    self.phase = phase

  def __len__(self):
    return len(self.file_list)

  def __getitem__(self, idx):
    img_path = self.file_list[idx]
    img = Image.open(img_path)
    img_transformed = self.transform(img)

    virtual_idx = img_path.find(self.phase) + len(self.phase) + 1
    label = img_path[virtual_idx : virtual_idx + 4]
    label_num = 0 if label == "ants" else 1

    return img_transformed, label_num

In [9]:
train_dataset = MyDataset(train_list, transform, "train")
val_dataset = MyDataset(val_list, transform, "val")

In [10]:
img, label = train_dataset[127]
print(img.shape)
print(label)
print(len(train_dataset))

torch.Size([3, 224, 224])
1
243


# 3.Tạo DataLoader

In [11]:
batch_size = 4

train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size, shuffle=True)
val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size, shuffle=False)

dataloader_dict = {"train": train_dataloader, "val": val_dataloader}

In [12]:
batch_iterator = iter(dataloader_dict["train"])
imgs, labels = next(batch_iterator)
print(imgs.shape)
print(labels)

torch.Size([4, 3, 224, 224])
tensor([1, 0, 1, 0])


# 4.Xây dựng Network

In [13]:
net = models.vgg16(pretrained=True)
net.classifier[6] = nn.Linear(in_features=4096, out_features=2)
net.train()

Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth


  0%|          | 0.00/528M [00:00<?, ?B/s]

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

# 6.Định nghĩa hàm loss

In [14]:
criterior = nn.CrossEntropyLoss()

# 7.Thiết lập thuật toán tối ưu

In [15]:
param_param = []
param_name = ["classifier.6.weight", "classifier.6.bias"]
for name, param in net.named_parameters():
  if name in param_name:
    param.requires_grad = True
    param_param.append(param)
    print(name)
  else:
    param.requires_grad = False

print(param_param)

classifier.6.weight
classifier.6.bias
[Parameter containing:
tensor([[ 0.0032, -0.0111, -0.0049,  ..., -0.0108,  0.0056,  0.0122],
        [ 0.0076, -0.0068,  0.0041,  ...,  0.0023,  0.0049,  0.0060]],
       requires_grad=True), Parameter containing:
tensor([0.0065, 0.0151], requires_grad=True)]


In [16]:
optimizer = optim.SGD(params=param_param, lr=0.001, momentum=0.9)

# 8.Thực hiện việc học và Kiểm thử network

In [17]:
def train(net, dataloader_dict, criterior, optimizer, num_epochs):
  device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
  print("device: ", device)
  print()
  for epoch in range(num_epochs):
    print("Epoch {}/{}:".format(epoch + 1, num_epochs))

    net.to(device)
    torch.backends.cudnn.benchmark = True

    for phase in ["train","val"]:
      if phase == "train":
        net.train()
      else:
        net.eval()
      
      epoch_loss = 0.0
      epoch_correct = 0

      for inputs, labels in tqdm(dataloader_dict[phase]):
        optimizer.zero_grad()
        inputs = inputs.to(device)
        labels = labels.to(device)

        with torch.set_grad_enabled(phase == "train"):
          outputs = net(inputs)

          # tính sum(loss) và sum(correct)
          loss = criterior(outputs, labels)
          epoch_loss += loss.item() * inputs.size(0)
          _, preds = torch.max(outputs, 1)
          epoch_correct += torch.sum(preds == labels)

          # từ single_loss -> update_params
          if phase == "train":
            loss.backward()
            optimizer.step()

      epoch_loss = epoch_loss / len(dataloader_dict[phase].dataset)
      epoch_accuracy = epoch_correct.double() / len(dataloader_dict[phase].dataset)

      print("{:<10} Loss: {:<10.4f} Acc: {:<10.4f}".format(phase, epoch_loss, epoch_accuracy))
    
    print()



In [20]:
num_epochs = 40
train(net, dataloader_dict, criterior, optimizer, num_epochs)

device:  cuda:0

Epoch 1/40:


100%|██████████| 61/61 [00:03<00:00, 18.44it/s]


train      Loss: 0.0550     Acc: 0.9835    


100%|██████████| 39/39 [00:02<00:00, 18.78it/s]


val        Loss: 0.0888     Acc: 0.9542    

Epoch 2/40:


...

val        Loss: 0.1251     Acc: 0.9477    

Epoch 40/40:


100%|██████████| 61/61 [00:03<00:00, 18.44it/s]


train      Loss: 0.0058     Acc: 1.0000    


100%|██████████| 39/39 [00:02<00:00, 18.38it/s]

val        Loss: 0.1302     Acc: 0.9608    



#9.Dự đoán với dataset

## 9.4. Tạo Predictor

### 9.4.1. Tạo class Predictor

In [ ]:
class Predictor():
  def __init__(self, class_index):
    self.class_index = class_index

  def predict(self, out):
    maxid = np.argmax(out.detach().numpy())
    name = self.class_index[str(maxid)]
    return name

### 9.4.2. Tạo class_index dictionary

In [ ]:
class_index = {"0": "ant", "1": "bee"}

### 9.4.3. Tạo máy predictor

In [ ]:
predictor = Predictor(class_index)

## 9.5. Ví dụ Predict

In [ ]:
img = Image.open("/content/Bee_Collecting_Pollen_2004-08-14.jpg")
img_transformed = transform(img, "val")
img_transformed = img_transformed.unsqueeze_(0)
output = net(img_transformed)
name = predictor.predict(output)
print(name)


bee
